## Importation de bibliothèques

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from datetime import datetime
import warnings
import h2o
from h2o.estimators import h2oRandomForestEstimator


ModuleNotFoundError: No module named 'h2o'

## Chargement et pretraitement des données

In [37]:
train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
print(train_df.columns)
print(test_df.columns)

warnings.filterwarnings('ignore')

# Vérifier la présence de la colonne 'Embarked'
if 'Embarked' in train_df.columns:
    # Remplir les valeurs manquantes pour 'Embarked' avec la valeur la plus fréquente (si présente)
    train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
else:
    print("La colonne 'Embarked' n'est pas présente dans train_df")

if 'Embarked' in test_df.columns:
    test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)
else:
    print("La colonne 'Embarked' n'est pas présente dans test_df")

# Remplir les valeurs manquantes pour 'Age' avec la médiane
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)

# Remplir les valeurs manquantes pour 'Embarked' avec la valeur la plus fréquente
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)

# Remplir les valeurs manquantes pour 'Fare' dans le jeu de test avec la médiane
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

# Supprimer la colonne 'Cabin' car elle contient trop de valeurs manquantes
train_df.drop('Cabin', axis=1, inplace=True)
test_df.drop('Cabin', axis=1, inplace=True)

# Créer une nouvelle feature 'FamilySize' à partir de 'SibSp' et 'Parch'
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# Créer une nouvelle feature 'IsAlone' à partir de 'FamilySize'
train_df['IsAlone'] = 1
train_df.loc[train_df['FamilySize'] > 1, 'IsAlone'] = 0
test_df['IsAlone'] = 1
test_df.loc[test_df['FamilySize'] > 1, 'IsAlone'] = 0

# Supprimer les colonnes inutiles
train_df = train_df.drop(['Name', 'Ticket', 'SibSp', 'Parch'], axis=1)
test_df = test_df.drop(['Name', 'Ticket', 'SibSp', 'Parch'], axis=1)

# Convertir la feature 'Sex' en numérique
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1}).astype(int)
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1}).astype(int)

# Convertir la feature 'Embarked' en numérique en utilisant le one-hot encoding (à voir les tech)
train_df = pd.get_dummies(train_df, columns=['Embarked'])
test_df = pd.get_dummies(test_df, columns=['Embarked'])


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


## Séparer les données d'entraînement

In [38]:
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Démarrer un run MLflow et entraîner le modèle

In [39]:
train_h2o = h2o.H2OFrame(pd.concat([X, y], axis=1))

model = H2ORandomForestEstimator(ntrees=100, max_depth=20, seed=1)
model.train(x=X.columns.tolist(), y="Survived", training_frame=train_h2o)

NameError: name 'h2o' is not defined

## Évaluer le modèle et enregistrer les métriques dans MLflow

In [40]:
model_path = h2o.save_model(model=model, path="./", force=True)
print(f"Model saved to: {model_path}")

NameError: name 'h2o' is not defined

## Faire des prédictions sur le jeu de test et créer un fichier de soumission

In [41]:
# Faire des prédictions sur le jeu de test
test_h2o = h2o.H2OFrame(test_df)
predictions = model.predict(test_h2o).as_data_frame()["predict"]
# Créer un fichier de soumission
submission = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': predictions})
submission.to_csv('../Results/submission.csv', index=False)

NameError: name 'h2o' is not defined